# Getting Json Outputs from Llama3 using LangChain 

In [ ]:
!pip install -U langchain-ollama langchain langchain-core 

In [1]:
import json
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [2]:
json_schema = {
    "title": "Person",
    "description": "Identifying information about a person.",
    "type": "object",
    "properties": {
    "name": {"title": "Name", "description": "The person's name", "type": "string"},
    "age": {"title": "Age", "description": "The person's age", "type": "integer"},
    "favoriteFood": {
    "title": "Fav Food",
    "description": "The person's favorite food",
    "type": "string",
    "required": ["name", "age", "favoriteFood"], }
    }
}

In [11]:
# keep the model loaded indefinitely temperature=0.1,
llm = ChatOllama(model="llama3.1", keep_alive=-1, temperature=0)

messages = [
HumanMessage (content="Please tell me about a person using the following JSON schema:"),
HumanMessage (content="{schema}"),
HumanMessage (content="The person is named Lucas who is 17 years old and loves pancakes."),
]

In [12]:
prompt = ChatPromptTemplate.from_messages(messages)

In [13]:
prompt

ChatPromptTemplate(input_variables=[], messages=[HumanMessage(content='Please tell me about a person using the following JSON schema:'), HumanMessage(content='{schema}'), HumanMessage(content='The person is named Lucas who is 17 years old and loves pancakes.')])

In [14]:
# converting json schema to a string
schema = json.dumps(json_schema, indent=2)

In [15]:
chain = prompt | llm | JsonOutputParser()

In [16]:
response = chain.invoke({"schema": schema})

In [17]:
print(response)
print(type(response))

{'name': 'Lucas', 'age': 17, 'favoriteFood': 'pancakes'}
<class 'dict'>


We can turn this structured output functionality into a function calling pipeline.

In [18]:
def some_tool_that_prints_info_from_dict(json_dict):
    for key in json_dict.keys():
        print(f"{key}: {json_dict[key]}")

some_tool_that_prints_info_from_dict(response)

name: Lucas
age: 17
favoriteFood: pancakes
